# MSC_DA_CA1_Semester_2:
# A Comparative Study of VGG16 and EfficientNet for Potato Leaf Disease Detection: An Integration of Hadoop and Apache Spark

**Name:**
Madina Sagatova

**Student ID:**
2021255

**Student email:**
2021255@student.cct.ie

**Programme Title:**
MSc in Data Analytics

### Loading Image Dataset with PySpark

In [1]:
from pyspark.sql import SparkSession

# Create a Spark session 
spark = SparkSession.builder.appName('PotatoLeafDiseas').getOrCreate()

# Loading the images from HDFS 
data = spark.read.format('image').load('hdfs://localhost:9000/user1/potato_disease/*')

# Displaying schema to ensure the data is loaded correctly 
data.printSchema()

# Show some sample data
data.show()

24/09/27 21:28:02 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = true)
 |    |-- width: integer (nullable = true)
 |    |-- nChannels: integer (nullable = true)
 |    |-- mode: integer (nullable = true)
 |    |-- data: binary (nullable = true)



+--------------------+
|               image|
+--------------------+
|{hdfs://localhost...|
|{hdfs://localhost...|
|{hdfs://localhost...|
|{hdfs://localhost...|
|{hdfs://localhost...|
|{hdfs://localhost...|
|{hdfs://localhost...|
|{hdfs://localhost...|
|{hdfs://localhost...|
|{hdfs://localhost...|
|{hdfs://localhost...|
|{hdfs://localhost...|
|{hdfs://localhost...|
|{hdfs://localhost...|
|{hdfs://localhost...|
|{hdfs://localhost...|
|{hdfs://localhost...|
|{hdfs://localhost...|
|{hdfs://localhost...|
|{hdfs://localhost...|
+--------------------+
only showing top 20 rows



### Label & Preprocess Data 

In [2]:
from pyspark.sql.functions import regexp_extract, col

# Add a label column based on the 3 folder names: Potato___Early_blight, Potato___healthy, Potato___Late_blight
data = data.withColumn('label',regexp_extract(col('image.origin'), 'potato_disease/([^/]+)', 1))

# Show some labeled data
data.select('image.origin', 'label').show(15)

+--------------------+--------------------+
|              origin|               label|
+--------------------+--------------------+
|hdfs://localhost:...|Potato___Early_bl...|
|hdfs://localhost:...|Potato___Early_bl...|
|hdfs://localhost:...|    Potato___healthy|
|hdfs://localhost:...|    Potato___healthy|
|hdfs://localhost:...|    Potato___healthy|
|hdfs://localhost:...|Potato___Early_bl...|
|hdfs://localhost:...|    Potato___healthy|
|hdfs://localhost:...|    Potato___healthy|
|hdfs://localhost:...|Potato___Early_bl...|
|hdfs://localhost:...|    Potato___healthy|
|hdfs://localhost:...|    Potato___healthy|
|hdfs://localhost:...|    Potato___healthy|
|hdfs://localhost:...|Potato___Early_bl...|
|hdfs://localhost:...|Potato___Early_bl...|
|hdfs://localhost:...|    Potato___healthy|
+--------------------+--------------------+
only showing top 15 rows

